# Question Answering using Embeddings

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the Nusring Fundamentals.

In [71]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import time
import os

openai.api_key = os.getenv('OPEN_AI_KEY') #Add your API key here

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [72]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Marcelo Chierighini of Brazil won the gold medal in the men's high jump at the 2020 Summer Olympics."

In [73]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

In [74]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Gianmarco Tamberi and Mutaz Essa Barshim emerged as joint winners of the event.'

In [75]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('nursing_fundamental_finalwithtokens.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

477 rows in the data.


content  \
title                                              heading                                                                                                 
Safety 5.4 Culture of Safety                       Open Resources for Nursing (Open RN) In additio...  Safety 5.4 Culture of Safety Open Resources fo...   
Grief and Loss 17.6 Applying the Nursing Proces... Outcomes An overall goal for a patient who is a...  Grief and Loss 17.6 Applying the Nursing Proce...   
Mobility 13.3 Applying the Nursing Process         Diagnoses There are several nursing diagnoses r...  Mobility 13.3 Applying the Nursing Process Dia...   
Cognitive Impairments 6.4 Applying the Nursing ... Diagnoses Commonly used NANDA-I nursing diagnos...  Cognitive Impairments 6.4 Applying the Nursing...   
Comfort 11.2 Comfort Basic Concepts                Substance Abuse Among Nurses and Nursing Studen...  Comfort 11.2 Comfort Basic Concepts Substance ...   

                                                                                                       tokens  
title                                              heading                                                     
Safety 5.4 Culture of Safety                       Open Resources for Nursing (Open RN) In additio...     944  
Grief and Loss 17.6 Applying the Nursing Proces... Outcomes An overall goal for a patient who is a...     287  
Mobility 13.3 Applying the Nursing Process         Diagnoses There are several nursing diagnoses r...     304  
Cognitive Impairments 6.4 Applying the Nursing ... Diagnoses Commonly used NANDA-I nursing diagnos...     242  
Comfort 11.2 Comfort Basic Concepts                Substance Abuse Among Nurses and Nursing Studen...     255

In [76]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    time.sleep(7)
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [77]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [78]:
#document_embeddings = load_embeddings("embeddings.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

document_embeddings = compute_doc_embeddings(df)
embeddings_df = pd.DataFrame(document_embeddings)
embeddings_df.to_csv('embeddings.csv', index=False,header=True)

APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out.

In [ ]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('Scope of Practice 1.1. Scope of Practice Introduction', 'Learning Objectives Discuss nursing scope of practice and standards of care Compare various settings in which nurses work Describe contributions of interprofessional health care team members Describe levels of nursing education and the NCLEX Discuss basic legal considerations and ethics Outline professional nursing organizations Examine quality and evidence-based practice in nursing You are probably wondering, â\x80\x9cWhat is scope of practice? What does it mean for me and my nursing practice?â\x80\x9d Scope of practice is defined as services that a trained health professional is deemed competent to perform and permitted to undertake according to the terms of their professional nursing license.American Nurses Association. (n.d.). Scope of practice.  Nursing scope of practice provides a framework and structured guidance for activities one can perform based on their nursing license. As a nurse and a nursing student, is always im

In [ ]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [ ]:
order_document_sections_by_query_similarity("What is the definition of scope of practice in nursing and how does it provide a framework for nursing activities?", document_embeddings)[:5]

[(0.8972067555461246,
  ('Scope of Practice 1.1. Scope of Practice Introduction',
   'Learning Objectives Discuss nursing scope of practice and standards of care Compare various settings in which nurses work Describe contributions of interprofessional health care team members Describe levels of nursing education and the NCLEX Discuss basic legal considerations and ethics Outline professional nursing organizations Examine quality and evidence-based practice in nursing You are probably wondering, â\x80\x9cWhat is scope of practice? What does it mean for me and my nursing practice?â\x80\x9d Scope of practice is defined as services that a trained health professional is deemed competent to perform and permitted to undertake according to the terms of their professional nursing license.American Nurses Association. (n.d.). Scope of practice.  Nursing scope of practice provides a framework and structured guidance for activities one can perform based on their nursing license. As a nurse and a nu

In [ ]:
order_document_sections_by_query_similarity("Who won the women's high jump?", document_embeddings)[:5]

[(0.7220074131334102,
  ('Comfort 11.6 Putting It All Together',
   'Patient Scenario Mrs. Jamison is a 34-year-old woman admitted through the emergency department with kidney stones. As you reposition her in bed, she is visibly grimacing and audibly moaning. She rates her pain at an â\x80\x9c8 out of 10â\x80\x9d although she reports her pain has â\x80\x9cimprovedâ\x80\x9d since admission with the IV morphine delivered via PCA pump. You recheck her vital signs and her blood pressure is elevated at 150/90 and her heart rate is 120.')),
 (0.7218717329890921,
  ('Mobility 13.4 Putting It All Together',
   'Patient Scenario Mrs. Howard is a 73-year-old woman who was recently admitted to the medical surgical floor with pneumonia. She has an underlying history of emphysema and has experienced a recent exacerbation in dyspnea during activity. This morning when being assisted to the bathroom, she reports, â\x80\x9cI have to stop and catch my breath when walking.â\x80\x9d Vital signs this morni

We can see that the most relevant document sections for each question include the summaries for the Men's and Women's high jump competitions - which is exactly what we would expect.

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [ ]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [ ]:
prompt = construct_prompt(
    "What are Basic Fluid and Electrolyte Concepts?",
    document_embeddings,
    df
)

print("===\n", prompt)

KeyError: 'Fluid Imbalance Two types of fluid imbalances are excessive fluid volume (also referred to as hypervolemia) and deficient fluid volume (also referred to as hypovolemia). These imbalances primarily refer to imbalances in the extracellular compartment, but can cause fluid movement in the intracellular compartments based on the sodium level of the blood. Excessive Fluid Volume Excessive fluid volume (also referred to as hypervolemia) occurs when there is increased fluid retained in the intravascular compartment. Patients at risk for developing excessive fluid volume are those with the following conditions: Heart Failure Kidney Failure Cirrhosis PregnancyLewis, J. L., III. (June 2020). Volume overload. Merck Manual Professional Version.  Symptoms of fluid overload include pitting edema, ascites, and dyspnea and crackles from fluid in the lungs. Edema is swelling in dependent tissues due to fluid accumulation in the interstitial spaces. Ascites is fluid retained in the abdomen. T

In [ ]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [81]:
answer_query_with_context("What is Sleep & Rest Introduction?", df, document_embeddings)

KeyError: 'Learning Objectives Assess factors that put patients at risk for problems with sleep Identify factors related to sleep/rest across the life span Recognize characteristics of sleep deprivation Consider the use of nonpharmacological measures to promote sleep and rest Identify evidence-based practices Maslowâ\x80\x99s hierarchy of needs indicates sleep as one of our physiological requirements. Getting enough quality sleep at the right times according to our circadian rhythms can protect mental and physical health, safety, and quality of life. Conversely, chronic sleep deficiency increases the risk of heart disease, kidney disease, high blood pressure, diabetes, and stroke, as well as weakening the immune system.Trossman, S. (2018, November 7). Nurses offer strategies to promote patientsâ\x80\x99 rest and sleep. American Nurse.  This chapter will review the physiology of sleep and common sleep disorders, as well as interventions to promote good sleep.'

In [ ]:
query = "What is secondary care in the medical field and in what types of settings is it typically provided?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('Scope of Practice 1.4 Health Care Settings & Team', 'Health Care Settings There are several levels of health care including primary, secondary, and tertiary care. Each of these levels focuses on different aspects of health care and is typically provided in different settings. Primary Care Primary care promotes wellness and prevents disease. This care includes health promotion, education, protection (such as immunizations), early disease screening, and environmental considerations. Settings providing this type of health care include physician offices, public health clinics, school nursing, and community health nursing. Secondary care Secondary care occurs when a person has contracted an illness or injury and requires medical care. Secondary care is often referred to as acute care. Secondary care can range from uncomplicated care to repair a small laceration or treat a strep throat infection to more complicated emergent care such as treating a head injury 

In [ ]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Spirituality 18.6 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x99s office with complaints of difficulty concentrating, sadness, and anxiety. The patient recently experienced the loss of his wife in a motor vehicle accident and reports difficulty sleeping and weight loss of greater than 15 pounds in the previous month. He reports feeling â\x80\x9chopelessâ\x80\x9d and â\x80\x9cangry at Godâ\x80\x9d for the loss that he has experienced. He states he used to attend religious services with his wife, but â\x80\x9cThat was really more of â\x80\x98herâ\x80\x99 thing. I really donâ\x80\x99t know what to believe anymore.â\x80\x9d')
('Grief and Loss 17.7 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x99s office for a follow-up visit. The patient recently experienced the loss of his wife in a motor vehicle accident and reports, â\x80\x9cI have p

In [ ]:
query = "What was unusual about the men’s shotput competition?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Spirituality 18.6 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x99s office with complaints of difficulty concentrating, sadness, and anxiety. The patient recently experienced the loss of his wife in a motor vehicle accident and reports difficulty sleeping and weight loss of greater than 15 pounds in the previous month. He reports feeling â\x80\x9chopelessâ\x80\x9d and â\x80\x9cangry at Godâ\x80\x9d for the loss that he has experienced. He states he used to attend religious services with his wife, but â\x80\x9cThat was really more of â\x80\x98herâ\x80\x99 thing. I really donâ\x80\x99t know what to believe anymore.â\x80\x9d')
('Grief and Loss 17.7 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x99s office for a follow-up visit. The patient recently experienced the loss of his wife in a motor vehicle accident and reports, â\x80\x9cI have p

In [ ]:
query = "In the 2020 Summer Olympics, how many silver medals did Italy win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 3 document sections:
('Sleep and Rest 12.4 Putting It All Together', 'Patient Scenario Mrs. Salvo is a 65-year-old woman admitted to the hospital for a gastrointestinal (GI) bleed. She has been hospitalized for three days on the medical surgical floor. During this time, she has received four units of PRBCs, has undergone a colonoscopy, upper GI series, and had hemoglobin levels drawn every four hours. The nurse reports to the patientâ\x80\x99s room to conduct an assessment prior to beginning the 11 p.m.-7 a.m. shift. Although Mrs. Salvoâ\x80\x99s hemoglobin has stabilized for the last 24 hours, Mrs. Salvo appears fatigued with bags under her eyes. In conversation with her, she yawns frequently and wanders off in her train of thought. She reports, â\x80\x9cYou canâ\x80\x99t get any rest in here. I am poked and prodded at least once an hour.â\x80\x9d')
('Spirituality 18.6 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x9

In [ ]:
query = "What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Spirituality 18.6 Putting It All Together', 'Patient Scenario Mr. Yun is a 34-year-old man presenting to his physicianâ\x80\x99s office with complaints of difficulty concentrating, sadness, and anxiety. The patient recently experienced the loss of his wife in a motor vehicle accident and reports difficulty sleeping and weight loss of greater than 15 pounds in the previous month. He reports feeling â\x80\x9chopelessâ\x80\x9d and â\x80\x9cangry at Godâ\x80\x9d for the loss that he has experienced. He states he used to attend religious services with his wife, but â\x80\x9cThat was really more of â\x80\x98herâ\x80\x99 thing. I really donâ\x80\x99t know what to believe anymore.â\x80\x9d')
('Nutrition 14.4 Putting It All Together', 'Patient Scenario Mr. Curtis is a 47-year-old patient admitted to the hospital with increased weakness, fatigue, and dehydration. His skin appears dry, and tenting occurs when skin turgor is evaluated. He is currently undergoing che